In [2]:
from sklearn import metrics

In [3]:
import pandas as pd

2. Given the following confusion matrix, evaluate (by hand) the model's performance.

    |               | pred dog   | pred cat   |
    |:------------  |-----------:|-----------:|
    | actual dog    |         46 |         7  |
    | actual cat    |         13 |         34 |



- In the context of this problem, what is a false positive? **Incorrectly predicting dog**
- In the context of this problem, what is a false negative? **Incorrectly predicting cat**
- How would you describe this model?
    - Model Accuracy = 80%
    - Recall = 86.8%
    - Precision = 78%

In [4]:
TP = 46
TN = 34
FP = 13
FN = 7

accuracy = (TP + TN) / (TP + TN + FP + FN)
recall = TP / (TP + FN)
precision = TP / (TP + FP)
accuracy, recall, precision

(0.8, 0.8679245283018868, 0.7796610169491526)

---
3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

    Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

    Use the predictions dataset and pandas to help answer the following questions:

    - An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?
    - Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

See predictions

In [69]:
preds = pd.read_csv('data/c3.csv')
preds.head(3)

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect


Get baseline model

In [70]:
display(preds['actual'].value_counts())
target_mode = preds['actual'].value_counts().index[0]
target_mode

No Defect    184
Defect        16
Name: actual, dtype: int64

'No Defect'

In [71]:
def get_baseline_model(df, target):
    '''
    Takes in a DataFrame and target column
    Makes target mode its prediction
    Returns df with baseline model predictions
    '''
    # find most frequent target class
    target_mode = df[target].value_counts().index[0]
    # predict target mode everytime
    df['model_baseline'] = target_mode
    
    return df

Apply function

In [72]:
preds = get_baseline_model(preds, target='actual')
# verify model_baseline value
print(preds['model_baseline'].unique())
preds.head(3)

['No Defect']


,actual,model1,model2,model3,model_baseline
0,No Defect,No Defect,Defect,No Defect,No Defect
1,No Defect,No Defect,Defect,Defect,No Defect
2,No Defect,No Defect,Defect,No Defect,No Defect


Manually get accuracy score

In [73]:
conf_matrix = metrics.confusion_matrix(preds['actual'], preds['model_baseline'], labels=['Defect','No Defect'])
conf_matrix

array([[  0,  16],
       [  0, 184]])

In [74]:
TP = conf_matrix[0][0]
TN = conf_matrix[1][1]
FP = conf_matrix[1][0]
FN = conf_matrix[0][1]

TP, TN, FP, FN

(0, 184, 0, 16)

In [75]:
accuracy = (TP + TN) / (TP + TN + FP + FN)
accuracy

0.92

In [76]:
metrics.accuracy_score(preds['actual'], preds['model_baseline'])

0.92

In [77]:
print(metrics.classification_report(preds['actual'], preds['model_baseline'], labels=['Defect', 'No Defect'], 
                                      zero_division=0))

              precision    recall  f1-score   support

      Defect       0.00      0.00      0.00        16
   No Defect       0.92      1.00      0.96       184

    accuracy                           0.92       200
   macro avg       0.46      0.50      0.48       200
weighted avg       0.85      0.92      0.88       200



An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible.
- Which evaluation metric would be appropriate here? 
    - Because we want to identify as many ducks as we can, we are ok accidentally classifying non-defective ducks as defective (FP). Thus, we will try to maximize recall, which calculates proportion of defective ducks we were able to predict.
- Which model would be the best fit for this use case?
    - Model 3

In [88]:
models = ['model1','model2','model3','model_baseline']

for model in models:
    recall = metrics.recall_score(preds['actual'], preds[model], pos_label='Defect')
    print(f'{model} recall score: {recall}')
    report = metrics.classification_report(preds['actual'], preds[model], labels=['Defect', 'No Defect'], 
                                           zero_division=0)
    print(f'{report}')  

model1 recall score: 0.5
              precision    recall  f1-score   support

      Defect       0.80      0.50      0.62        16
   No Defect       0.96      0.99      0.97       184

    accuracy                           0.95       200
   macro avg       0.88      0.74      0.79       200
weighted avg       0.95      0.95      0.94       200

model2 recall score: 0.5625
              precision    recall  f1-score   support

      Defect       0.10      0.56      0.17        16
   No Defect       0.94      0.56      0.70       184

    accuracy                           0.56       200
   macro avg       0.52      0.56      0.44       200
weighted avg       0.87      0.56      0.66       200

model3 recall score: 0.8125
              precision    recall  f1-score   support

      Defect       0.13      0.81      0.23        16
   No Defect       0.97      0.53      0.69       184

    accuracy                           0.56       200
   macro avg       0.55      0.67      0.46    

- Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you they really don't want to accidentally give out a vacation package when the duck really doesn't have a defect.
- Which evaluation metric would be appropriate here? 
    - Classifying a duck as defective when it is not (FP) is costly here. So we want to use precision in order to minimize false positives.
- Which model would be the best fit for this use case?
    - Model 1

In [91]:
for model in models:
    recall = metrics.precision_score(preds['actual'], preds[model], pos_label='Defect', zero_division=0)
    print(f'{model} recall score: {recall}')
    report = metrics.classification_report(preds['actual'], preds[model], labels=['Defect', 'No Defect'], 
                                           zero_division=0)
    print(f'{report}')  

model1 recall score: 0.8
              precision    recall  f1-score   support

      Defect       0.80      0.50      0.62        16
   No Defect       0.96      0.99      0.97       184

    accuracy                           0.95       200
   macro avg       0.88      0.74      0.79       200
weighted avg       0.95      0.95      0.94       200

model2 recall score: 0.1
              precision    recall  f1-score   support

      Defect       0.10      0.56      0.17        16
   No Defect       0.94      0.56      0.70       184

    accuracy                           0.56       200
   macro avg       0.52      0.56      0.44       200
weighted avg       0.87      0.56      0.66       200

model3 recall score: 0.13131313131313133
              precision    recall  f1-score   support

      Defect       0.13      0.81      0.23        16
   No Defect       0.97      0.53      0.69       184

    accuracy                           0.56       200
   macro avg       0.55      0.67    

---
4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

    At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

    Several models have already been developed with the data, and you can find their results in `data/gives_you_paws.csv`.

In [98]:
preds = pd.read_csv('data/gives_you_paws.csv')
preds.head(3)

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog


Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class).


In [99]:
# check class counts
preds['actual'].value_counts(normalize=True)

dog    0.6508
cat    0.3492
Name: actual, dtype: float64

In [100]:
preds['model_baseline'] = 'dog'
preds.head(3)

,actual,model1,model2,model3,model4,model_baseline
0,cat,cat,dog,cat,dog,dog
1,dog,dog,cat,cat,dog,dog
2,dog,cat,cat,cat,dog,dog


In [101]:
display(preds['actual'].value_counts())
target_mode = preds['actual'].value_counts().index[0]
target_mode

dog    3254
cat    1746
Name: actual, dtype: int64

'dog'

a. 

In terms of accuracy, how do the various models compare to the baseline model?
- models 1 and 4 have better accuracy while models 2 and 3 don't.

Are any of the models better than the baseline?
- **model1 and model4**

In [102]:
preds.columns[1:]

Index(['model1', 'model2', 'model3', 'model4', 'model_baseline'], dtype='object')

In [103]:
for model in preds.columns[1:]:
    accuracy = metrics.accuracy_score(preds['actual'], preds[model])
    print(f'{model} accuracy: {accuracy}')
    report = metrics.classification_report(preds['actual'], preds[model], labels=['dog', 'cat'],
                                           zero_division=0)
    print(f'{report}')

model1 accuracy: 0.8074
              precision    recall  f1-score   support

         dog       0.89      0.80      0.84      3254
         cat       0.69      0.82      0.75      1746

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000

model2 accuracy: 0.6304
              precision    recall  f1-score   support

         dog       0.89      0.49      0.63      3254
         cat       0.48      0.89      0.63      1746

    accuracy                           0.63      5000
   macro avg       0.69      0.69      0.63      5000
weighted avg       0.75      0.63      0.63      5000

model3 accuracy: 0.5096
              precision    recall  f1-score   support

         dog       0.66      0.51      0.57      3254
         cat       0.36      0.51      0.42      1746

    accuracy                           0.51      5000
   macro avg       0.51      0.51      0.50      5000
we

b. 

Suppose you are working on a team that solely deals with dog pictures.

Which of these models would you recommend?

- I want to minimize incorrectly labeling a picture as 'dog' (false positive), so I will focus on maximizing precision. 


c.

Suppose you are working on a team that solely deals with cat pictures.

Which of these models would you recommend?

- I want to minimize incorrectly labeling a picture as a cat


---
5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

    `sklearn.metrics.accuracy_score`

    `sklearn.metrics.precision_score`

    `sklearn.metrics.recall_score`

    `sklearn.metrics.classification_report`
